In [12]:
import os
from pathlib import Path
import pandas as pd

In [18]:
project_dir = "C:\\Users\\pyramidheadshark\\Repos\\Steam-AI-Recommendations"
file_path = os.path.join(project_dir, 'data', 'raw', 'Steam Games 2024.csv')
file_path

'C:\\Users\\pyramidheadshark\\Repos\\Steam-AI-Recommendations\\data\\raw\\Steam Games 2024.csv'

In [ ]:
df = pd.read_csv(file_path)

C:\Users\pyramidheadshark\AppData\Local\Temp\ipykernel_18832\2618208116.py:1: DtypeWarning: Columns (0,4,5,6,7,8,17,18,19,20,22,23,24,25,26,27,29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97428 entries, 0 to 97427
Data columns (total 40 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   AppID                       97415 non-null  object
 1   Name                        97412 non-null  object
 2   Release date                97418 non-null  object
 3   Estimated owners            97417 non-null  object
 4   Peak CCU                    97417 non-null  object
 5   Required age                97417 non-null  object
 6   Price                       97416 non-null  object
 7   Discount                    97417 non-null  object
 8   DLC count                   97418 non-null  object
 9   About the game              92537 non-null  object
 10  Supported languages         97410 non-null  object
 11  Full audio languages        97410 non-null  object
 12  Reviews                     10132 non-null  object
 13  Header image                97409 non-null  ob

In [22]:
df.columns

Index(['AppID', 'Name', 'Release date', 'Estimated owners', 'Peak CCU',
       'Required age', 'Price', 'Discount', 'DLC count', 'About the game',
       'Supported languages', 'Full audio languages', 'Reviews',
       'Header image', 'Website', 'Support url', 'Support email', 'Windows',
       'Mac', 'Linux', 'Metacritic score', 'Metacritic url', 'User score',
       'Positive', 'Negative', 'Score rank', 'Achievements', 'Recommendations',
       'Notes', 'Average playtime forever', 'Average playtime two weeks',
       'Median playtime forever', 'Median playtime two weeks', 'Developers',
       'Publishers', 'Categories', 'Genres', 'Tags', 'Screenshots', 'Movies'],
      dtype='object')